# Keras Neural Net

Now we'll change our model.  Originally it was defined using tf.contrib.layers, we'll change that to Keras.  Keras is being integrated as a first class citizen in TensorFlow and we'll use the version of Keras in tf.contrib.keras.  Originally the code looked like the following.

```
first_hidden_layer = tf.contrib.layers.relu(features, 10)

# Connect the second hidden layer to first hidden layer with relu
second_hidden_layer = tf.contrib.layers.relu(first_hidden_layer, 10)

# Connect the output layer to second hidden layer (no activation fn)
output_layer = tf.contrib.layers.linear(second_hidden_layer, 1)
```

The Keras version of the model looks like the following.

```
# Define our model in Keras
first_hidden_layer = Dense(10, activation='relu')(features)
second_hidden_layer = Dense(10, activation='relu')(first_hidden_layer)
output_layer = Dense(1, activation='linear')(second_hidden_layer)
```

Both of these models are similarly concise because the aim of both is to provide an abstraction from raw TensorFlow.  The point of this solution is not to propose Keras as either superior or inferior to tf.contrib.learn or tf.contrib.layers.  The point is that it is enormously powerful and benefical to (a) be able to access Keras from the TensorFlow distribution and (b) to be able to mix and match tf.contrib.learn, tf.contrib.layers and Keras in the same model.

You should familiarize yourself with the changes made, run this notebook in its entirety, then proceed to the [next notebook](5-keras-full.ipynb).

The output from training and evaluation will be the same as before (N.B. because of the nature of neural net training the actual numbers will be similar but not exactly the same.

In [1]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""DNNRegressor with custom estimator for abalone dataset."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import numpy as np

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

from tensorflow.contrib.keras.python.keras.layers import Dense

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
TRAINING_FILENAME = 'abalone_train.csv'
TEST_FILENAME = 'abalone_test.csv'
  
LEARNING_RATE = 0.001

In [3]:
training_dataset = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=TRAINING_FILENAME, target_dtype=np.int, features_dtype=np.float32)
test_dataset = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=TEST_FILENAME, target_dtype=np.int, features_dtype=np.float32)

In [4]:
def model_fn(features, targets, mode, params):
  
  # Define our model in Keras
  first_hidden_layer = Dense(10, activation='relu')(features)
  second_hidden_layer = Dense(10, activation='relu')(first_hidden_layer)
  output_layer = Dense(1, activation='linear')(second_hidden_layer)

  # Reshape output layer to 1-dim Tensor to return predictions
  predictions = tf.reshape(output_layer, [-1])
  predictions_dict = {"ages": predictions}

  # Calculate loss using mean squared error
  loss = tf.losses.mean_squared_error(targets, predictions)

  # Calculate root mean squared error as additional eval metric
  eval_metric_ops = {
      "rmse": tf.metrics.root_mean_squared_error(
          tf.cast(targets, tf.float32), predictions)
  }

  train_op = tf.contrib.layers.optimize_loss(
      loss=loss,
      global_step=tf.contrib.framework.get_global_step(),
      learning_rate=params["learning_rate"],
      optimizer="SGD")

  return model_fn_lib.ModelFnOps(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops)

In [5]:
# Set model params
model_params = {"learning_rate": LEARNING_RATE}

# Instantiate Estimator
nn = tf.contrib.learn.Estimator(model_fn=model_fn, params=model_params)

def get_train_inputs():
  x = tf.constant(training_dataset.data)
  y = tf.constant(training_dataset.target)
  return x, y

# Fit
nn.fit(input_fn=get_train_inputs, steps=5000)

# Score accuracy
def get_test_inputs():
  x = tf.constant(test_dataset.data)
  y = tf.constant(test_dataset.target)
  return x, y

ev = nn.evaluate(input_fn=get_test_inputs, steps=1)
print("Loss: %s" % ev["loss"])
print("Root Mean Squared Error: %s" % ev["rmse"])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4b7a4bed0>, '_model_dir': '/tmp/tmpzBe3c_', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpzBe3c_/model.ckpt.
INFO:tensorflow:loss = 140.183, step = 1
INFO:tensorflow:global_step/sec: 610.27
INFO:tensorflow:loss = 8.59873, step = 101 (0.167 sec)
INFO:tensorflow:global_step/sec: 632.638
INFO:tensorflow:loss = 7.41668, step = 201 (0.158 sec)
INFO